In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install youtokentome

     |████████████████████████████████| 1.7 MB 5.2 MB/s 


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

import youtokentome as yttm
import numpy as np

import random
import math
import time
import pickle
from tqdm.auto import tqdm

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [ ]:
# token_model = yttm.BPE(model='try_poems_embed_yttm.model', n_threads=-1)
token_model = yttm.BPE(model='/content/drive/MyDrive/Colab Notebooks/NLP_poems/voc10k_poems_embed_yttm.model', n_threads=-1)

In [ ]:
class newDataset(Dataset):
    def __init__(self, data_list_of_list):
        self.data = data_list_of_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def cut_length(self):
#         middle_len = sum([len(text) for text in self.data])/len(self.data)
#         limit = int(middle_len)*2
        limit = 10
        self.data = [text if len(text)<limit else text[:limit] for text in self.data]

    def distribute_data(self, share_tr, share_val):  # examle = (0.8, 0.1)
        rand_i = np.random.permutation(len(self.data))
        n1 = int(len(self.data) * share_tr)
        n2 = int(len(self.data) * share_val)
        # return self.data[rand_i[0: n1]], self.data[rand_i[n1: n1 + n2]], self.data[rand_i[n1 + n2:]]
        return [self.data[i] for i in rand_i[0: n1]], \
               [self.data[i] for i in rand_i[n1: n1 + n2]], \
               [self.data[i] for i in rand_i[n1 + n2:]]
    
def padding(batch):
    pad_id = token_model.subword_to_id('<PAD>')
    
    batch = [torch.tensor(x) for x in batch]
    batch = pad_sequence(batch, batch_first=False, padding_value=pad_id)  # batch_first=True -> [batch size, len]
    return batch
    # return batch.to(device)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers) # dropout = dropout
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):      
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        return hidden

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim  # the size of the vocabulary for the output/target.
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim) # 1 - size of dict emb, 2 - size of emb vec
        
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers) # dropout = dropout
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden):
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        
        #input = [batch size]
        input = input.unsqueeze(0) 
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        #embedded = [1, batch size, emb dim]
                
        output, hidden = self.rnn(embedded, hidden)
        #output = [1, batch size, hid dim], seq len will always be 1 in the decoder
        #hidden = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.2):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio=0.75 -> use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder = the initial hidden state of the decoder
        hidden = self.encoder(src)
        
        #first input to the decoder is the <bos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden states
            #receive output tensor (predictions) and new hidden states
            output, hidden = self.decoder(input, hidden)
            
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token, else use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [ ]:
INPUT_DIM = token_model.vocab_size() 
OUTPUT_DIM = INPUT_DIM
ENC_EMB_DIM = 128 #256
DEC_EMB_DIM = 128 #256
HID_DIM = 256 #512
N_LAYERS = 2
ENC_DROPOUT = 0.2 
DEC_DROPOUT = 0.2

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
PAD_ID = token_model.subword_to_id('<PAD>')
criterion = nn.CrossEntropyLoss(ignore_index = PAD_ID)

In [ ]:
def test(model, loader, criterion):
    
    model.eval()
    
    epoch_loss = 0
    outputs_list = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(tqdm(loader)):
            batch = batch.to(device, non_blocking=True)
            src = batch
            trg = batch

            output = model(src, trg, 0) #turn off teacher forcing
            outputs_list.append(torch.transpose(output, 0, 1))

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
      
    return epoch_loss / len(loader), outputs_list

In [ ]:
# optimizer.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/NLP_poems/b7e10dr03_optim.pt'))
# print(optimizer.param_groups[0]['lr'])
# print(optimizer.param_groups[0]['initial_lr'])
# print(optimizer.param_groups)

In [ ]:
# checkpoint = torch.load('trylr005_17k_checkpoint.pt')
checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/NLP_poems/AdamW01_C005_drEm2_voc10k_tf2_len10_Att_checkp_last.pt', 
                        map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
# with open('data/try_test_data_token.pickle', 'rb', ) as file:
with open('/content/drive/MyDrive/Colab Notebooks/NLP_poems/data/voc10klen20_test_data_token.pickle', 'rb', ) as file:
    test_data_token = pickle.load(file)
test_loader_load = DataLoader(
    test_data_token, batch_size=128, shuffle=False, num_workers=1, pin_memory=True, collate_fn=padding)

In [ ]:
test_loss, test_poems_embed = test(model, test_loader_load, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

  0%|          | 0/14 [00:00<?, ?it/s]

| Test Loss: 4.170 | Test PPL:  64.719 |


#### Get poems from test output and compare with orig test poems

In [ ]:
generated_poems_token = []
generated_poems_len = []
PAD_ID = token_model.subword_to_id('<PAD>')
BOS_ID = token_model.subword_to_id('<BOS>')
EOS_ID = token_model.subword_to_id('<EOS>')
for batch in test_poems_embed:
    for text_word_probab in batch:
        text_vocab_ids = [torch.argmax(word).item() for word in text_word_probab]
        generated_poems_token.append(text_vocab_ids)
        generated_poems_len.append(len(text_vocab_ids))

generated_poems = token_model.decode(generated_poems_token, ignore_ids=[PAD_ID]) 

orig_poems = token_model.decode(test_data_token, ignore_ids=[PAD_ID, BOS_ID, EOS_ID]) 

In [ ]:
for id in range(0, 30):
  # print(generated_poems_len[id])
  # print(len(orig_poems[id].split(' ')))
  print(generated_poems[id])
  print(orig_poems[id])
  print()

если надо уносижать не наруслав полй . не не не не не не не не
если надо уезжать не толпитесь стадом . политтройка не спеша всех

над самой бездной на крыль уздой железной избу огне и и и и и и и и и
над самой бездной на высоте уздой железной россию поднял на дыбы пушкин город змеи и медного

на губках смех в сердечке благодать которую печальный в страным иною и иною и
на губках смех в сердечке благодать которую ни светских правил стужа ни мненья

только русский знавший с детства тристь уст природа от предмия пред предмими с
только русский знавший с детства тяжесть вечной духоты с жизнью ваявший как наслед

я инок темный нищ и гол . я я я я я я я я я
я инок темный нищ и гол мне был глагол как гром огромный когда качая

упала лепетса и снова сурово разъяты закаром в в в и и и и в и
упала завеса и снова сурово разъяты закаты в горбатые старые скаты и в

посв . а курсинскому летит и и на на на на на на на на
посв . а курсинскому летит земля на крыльях духа тьмы летит навстречу солнцу и

б

#### Use metric to evaluate results

In [ ]:
embedding = nn.Embedding(OUTPUT_DIM, DEC_EMB_DIM) # 1 - size of dict emb, 2 - size of emb vec

embed_source_poems = embedding(torch.tensor(generated_poems_token)) # [poems num, word num, embed len]
embed_target_poems = embedding(torch.tensor(test_data_token))

ValueError: ignored

In [ ]:
total_score = 0
for i in range(len(embed_target_poems)):
    embed_source_poem = embed_source_poems[i]  # [word num, embed len]
    embed_target_poem = embed_target_poems[i]
    # print(embed_source_poem.shape , embed_target_poem.shape)
    # print(embed_source_poem, embed_target_poem)

    # [min, mean, max] = sentence emb-g
    v_s_min = torch.min(embed_source_poem)
    v_s_mean = torch.mean(embed_source_poem)
    v_s_max = torch.max(embed_source_poem)

    v_t_min = torch.min(embed_target_poem)
    v_t_mean = torch.mean(embed_target_poem)
    v_t_max = torch.max(embed_target_poem)

    v_s = torch.tensor([v_s_min, v_s_mean, v_s_max])
    v_t = torch.tensor([v_t_min, v_t_mean, v_t_max])
    # print(v_s, v_t)

    score = torch.matmul(v_s.t(), v_t) / (torch.linalg.norm(v_s, ord=2) * torch.linalg.norm(v_t, ord=2))
    # if source = orig, then score = 1.0, else score < 1.0
    total_score += (1.0 - score)
    
total_score = total_score.item()
print(total_score)

2.2617645263671875


In [ ]:
# print(token_model.vocab()[:30])
token_model.subword_to_id(' \n')
token_model.encode('где бы ты не был \n в каких бы краях . \n я бы всегда ')
token_model.encode('где бы ты не был \n в каких бы краях . \n я бы всегда ', output_type=yttm.OutputType.SUBWORD)

['▁где',
 '▁бы',
 '▁ты',
 '▁не',
 '▁был',
 '▁в',
 '▁каких',
 '▁бы',
 '▁краях',
 '▁.',
 '▁я',
 '▁бы',
 '▁всегда']

In [ ]:
a = torch.tensor(
    [
     [[1,2],[3,4],[5,6],[7,8]],
     [[0,0],[1,1],[2,2],[3,3]],
     [[-1,-1],[-2,-2],[-3,-3],[-4,-4]]
    ]
)
print(a.shape) # [3, 4, 2]
# в каждом стихе 3 слова, всего 4 стиха, каждое слово из 2х букв

a = torch.transpose(a, 0, 1)
print(a.shape) # [4, 3, 2]
print(a)